# RAG : Usando AI Search como base de conocimiento

In [ ]:
import os

In [46]:
!pip install -q langchain-community==0.3.8 faiss-cpu pypdf==5.1.0 langchain_openai==0.2.9


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
os.environ["AZURE_OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoi-datacitizen-practices.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "3lwQJSV6WUtsnkAqyv4350hfstiHYbaIYvHlKuhSlXMjN7sOScM3JQQJ99AKACYeBjFXJ3w3AAABACOGoi14"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"

In [ ]:
deployment_embedding = "text-embedding-3-small"
deployment_llm = "gpt-4o-mini"

## Generando fragmentos

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("pdf/ley-mineria-peru.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
    is_separator_regex = False
)

chunks = text_splitter.split_documents(data)

In [ ]:
len(chunks), chunks

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(azure_deployment = deployment_embedding)

In [ ]:
name = "dataleymineria" 

In [ ]:
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(chunks, embeddings)

# Save the FAISS vector database locally using the generated index path.
vectordb.save_local(name)

## Creando la base de conocimiento

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Creando la cadena de RAG

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, esta información no la tengo en mi base de conocimiento"

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

llm = AzureChatOpenAI(
    azure_deployment = deployment_llm,
    temperature = 0
)

prompt = PromptTemplate(
    input_variables = ["context", "question"], template = template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(rag_chain.invoke("¿Cuáles son las actividades mineras y cómo se clasifican?")))

In [ ]:
from IPython.display import Markdown, display
display(Markdown(rag_chain.invoke("¿Qué garantías y beneficios fiscales ofrece la ley para promover la inversión en minería?")))

In [ ]:
from IPython.display import Markdown, display
display(Markdown(rag_chain.invoke("qué es la IA Generativa")))